In [15]:
%pip install azureml-widgets
%pip install azureml-sdk[notebooks]

StatementMeta(64cad391-4c78-4947-a062-a4e8125f2168, 1, 54, Finished, Available, Finished)

You should consider upgrading via the '/nfs4/pyenv-252ff5f8-a40a-4474-a3f1-8ca6060b71c5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/nfs4/pyenv-252ff5f8-a40a-4474-a3f1-8ca6060b71c5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from azureml.core import Workspace, Experiment

# ws = Workspace.from_config()
subscription_name = "quick-starts-ws-266986"
subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30'

ws = Workspace.get(name = subscription_name,
subscription_id=subscription_id,
resource_group="aml-quickstarts-266986")


exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


run = exp.start_logging()

StatementMeta(64cad391-4c78-4947-a062-a4e8125f2168, 1, 60, Finished, Available, Finished)

Workspace name: quick-starts-ws-266986
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-266986


In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = subscription_name

vm_size = "Standard_D2_V2"
compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
#Create the compute cluster
compute_target = ComputeTarget.create(ws,cluster_name, compute_config)
#wait for the compute ready
compute_target.wait_for_completion(show_output=True)

StatementMeta(64cad391-4c78-4947-a062-a4e8125f2168, 1, 61, Finished, Available, Finished)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {"learning_rate": uniform(0.05, 0.1),
    "batch_size" : choice(16, 32, 64, 128)}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', 
file_path='Users/odl_user_266986/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    environment=sklearn_env,
    compute_target=compute_target
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="AUC",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=2
)

StatementMeta(64cad391-4c78-4947-a062-a4e8125f2168, 1, 62, Finished, Available, Finished)

In [19]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment = Experiment(workspace=ws, name="my-hyperdive-experiment")
experiment_run = experiment.submit(hyperdrive_config)
RunDetails(experiment_run).show()

StatementMeta(64cad391-4c78-4947-a062-a4e8125f2168, 1, 63, Finished, Available, Finished)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [20]:
import joblib
# Get your best run and save the model from that run.

best_run = experiment_run.get_best_run_by_primary_metric()
model= best_run.register_model(model_name="best_model",
model_path="outputs/best_model.pkl")

StatementMeta(64cad391-4c78-4947-a062-a4e8125f2168, 1, 64, Finished, Available, Finished)

AttributeError: 'NoneType' object has no attribute 'register_model'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###